# 6. 멀티 인덱스

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
titanic = sns.load_dataset('titanic')
df = titanic.loc[:,['age','sex','class','fare','survived']]

In [5]:
# class 열, sex 열 기준으로 분할
grouped = df.groupby(['class','sex'])

<bound method GroupBy.head of <pandas.core.groupby.generic.DataFrameGroupBy object at 0x0000023C1A9724C0>>

In [4]:
# 그룹 객체에 연산 메소드 적용
gdf = grouped.mean()
print(gdf)
print('\n')
print(type(gdf))

                     age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447


<class 'pandas.core.frame.DataFrame'>


In [6]:
# class 값이 First인 행을 선택하여 출력
gdf.loc['First']

,age,fare,survived
sex,,,
female,34.611765,106.125798,0.968085
male,41.281386,67.226127,0.368852


In [7]:
# class값이 First이고, sex값이 female인 행을 선택하여 출력
gdf.loc[('First','female')]

age          34.611765
fare        106.125798
survived      0.968085
Name: (First, female), dtype: float64

In [8]:
# sex 값이 male인 행을 선택하여 출력
# xs인덱서 

gdf.xs('male', level = 'sex')

,age,fare,survived
class,,,
First,41.281386,67.226127,0.368852
Second,30.740707,19.741782,0.157407
Third,26.507589,12.661633,0.135447


# 7. 피벗

In [9]:
import pandas as pd
import seaborn as sns

# IPython 디스플레이 설정 변경
pd.set_option('display.max_columns', 10)  # 출력할 최대 열의 개수
pd.set_option('display.max_colwidth', 20) # 출력할 열의 너비

In [11]:
df.head()

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0


In [12]:
# 행, 열, 값, 집계에 사용할 열을 1개씩 지정 - 평균 집계
pdf1 = pd.pivot_table(df,               # 피벗할 데이터프레임
                     index = 'class',   # 행 위치에 들어갈 열
                     columns = 'sex',   # 열 위치에 들어갈 열
                     values = 'age',    # 데이터로 사용할 열
                     aggfunc = 'mean')  # 데이터 집계 함수
pdf1.head()

sex,female,male
class,,
First,34.611765,41.281386
Second,28.722973,30.740707
Third,21.750000,26.507589


In [13]:
# 값에 적용하는 집계 함수 2개 이상 지정 가능 - 생존율, 생존자 수 집계
pdf2 = pd.pivot_table(df,               # 피벗할 데이터프레임
                     index = 'class',   # 행 위치에 들어갈 열
                     columns = 'sex',   # 열 위치에 들어갈 열
                     values = 'survived',    # 데이터로 사용할 열
                     aggfunc = ['mean','sum'])  # 데이터 집계 함수
pdf2.head()

mean              sum     
sex       female      male female male
class                                 
First   0.968085  0.368852     91   45
Second  0.921053  0.157407     70   17
Third   0.500000  0.135447     72   47

In [14]:
# 행, 열, 값에 사용할 열을 2개 이상 지정 가능 - 평균 나이, 최대 요금 집계
pdf3 = pd.pivot_table(df,                        # 피벗할 데이터프레임
                     index = ['class','sex'],    # 행 위치에 들어갈 열
                     columns = 'survived',       # 열 위치에 들어갈 열
                     values = ['age','fare'],    # 데이터로 사용할 열
                     aggfunc = ['mean','max'])   # 데이터 집계 함수
pdf3.head()

mean                                      max        \
                     age                   fare               age         
survived               0          1           0           1     0     1   
class  sex                                                                
First  female  25.666667  34.939024  110.604167  105.978159  50.0  63.0   
       male    44.581967  36.248000   62.894910   74.637320  71.0  80.0   
Second female  36.000000  28.080882   18.250000   22.288989  57.0  55.0   
       male    33.369048  16.022000   19.488965   21.095100  70.0  62.0   
Third  female  23.818182  19.329787   19.773093   12.464526  48.0  63.0   

                                 
                 fare            
survived            0         1  
class  sex                       
First  female  151.55  512.3292  
       male    263.00  512.3292  
Second female   26.00   65.0000  
       male     73.50   39.0000  
Third  female   69.55   31.3875

In [15]:
pdf3.xs('First') # 행 인덱스가 First인 행을 선택

mean                                      max                \
                age                   fare               age          fare   
survived          0          1           0           1     0     1       0   
sex                                                                          
female    25.666667  34.939024  110.604167  105.978159  50.0  63.0  151.55   
male      44.581967  36.248000   62.894910   74.637320  71.0  80.0  263.00   

                    
                    
survived         1  
sex                 
female    512.3292  
male      512.3292

In [16]:
pdf3.xs(('First','female'))

            survived
mean  age   0            25.666667
            1            34.939024
      fare  0           110.604167
            1           105.978159
max   age   0            50.000000
            1            63.000000
      fare  0           151.550000
            1           512.329200
Name: (First, female), dtype: float64

In [17]:
pdf3.xs('male', level = 'sex') # 행 인덱스의 sex 레벨이 male인 행을 선택

mean                                    max                \
                age                  fare              age          fare   
survived          0          1          0          1     0     1       0   
class                                                                      
First     44.581967  36.248000  62.894910  74.637320  71.0  80.0  263.00   
Second    33.369048  16.022000  19.488965  21.095100  70.0  62.0   73.50   
Third     27.255814  22.274211  12.204469  15.579696  74.0  45.0   69.55   

                    
                    
survived         1  
class               
First     512.3292  
Second     39.0000  
Third      56.4958

In [18]:
pdf3.xs(("Second",'male'), level = [0,'sex']) # Second, male인 행을 선택, 0: class

mean                               max                  
                   age               fare            age        fare      
survived             0       1          0        1     0     1     0     1
class  sex                                                                
Second male  33.369048  16.022  19.488965  21.0951  70.0  62.0  73.5  39.0

In [19]:
pdf3.xs('mean', axis = 1)

age                   fare            
survived               0          1           0           1
class  sex                                                 
First  female  25.666667  34.939024  110.604167  105.978159
       male    44.581967  36.248000   62.894910   74.637320
Second female  36.000000  28.080882   18.250000   22.288989
       male    33.369048  16.022000   19.488965   21.095100
Third  female  23.818182  19.329787   19.773093   12.464526
       male    27.255814  22.274211   12.204469   15.579696

In [20]:
pdf3.xs(1, level = 'survived', axis = 1) # survived 레벨이 1인 데이터 선택

mean               max          
                     age        fare   age      fare
class  sex                                          
First  female  34.939024  105.978159  63.0  512.3292
       male    36.248000   74.637320  80.0  512.3292
Second female  28.080882   22.288989  55.0   65.0000
       male    16.022000   21.095100  62.0   39.0000
Third  female  19.329787   12.464526  63.0   31.3875
       male    22.274211   15.579696  45.0   56.4958

In [21]:
pdf3.xs(('max','fare',0), level = [0,1,2], axis = 1) # max, fare, survived = 0인 데이터 선택

max
                 fare
survived            0
class  sex           
First  female  151.55
       male    263.00
Second female   26.00
       male     73.50
Third  female   69.55
       male     69.55